In [4]:
import torch
import torch.nn as nn
import tiktoken
import gpt_model

In [5]:
tokenizer = tiktoken.get_encoding("gpt2")

In [6]:
batch = []

txt1 = "Every effort moves you"
# in honor of Halloween (a Ministry song from the 80s)
txt2 = "This day is Halloween"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[ 6109,  3626,  6100,   345],
        [ 1212,  1110,   318, 16032]])


In [7]:
model = GPTModel(GPT_CONFIG_124M)

logits = model(batch)
print("Output shape:", logits.shape)
print(logits)

NameError: name 'GPTModel' is not defined